<a href="https://colab.research.google.com/github/MatteoRigoni/MachineLearningPlayground/blob/master/BaseMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Tecniche machine learning

> Apprendimento supervisionato (abbiamo input e output)
*  Regressione (l'output è un valore continuo, una quantità)
*  Classificazione (l'output è un valore discreto)

> Apprendimento non supervisionato (abbiamo solo input)
*  Associations (trovare regole che descrivono una porzione grande di dati)
*  Clustering (si raggruppano i dati per proprietà comuni)

> Apprendimento semi-supervisionato
*  Utilizzo di molti esempi senza label e pochi esempi con label

> Apprendimento per rinforzo
*  Si basa sulla realizzazione di **agenti intelligenti** in grado di prendere decisioni ed eseguire **azioni** in uno specifico **ambiente** al fine di massimizzare un **reward** e raggiungere un **obiettivo**


## Dataset

In [ ]:
import pandas as pd

In [ ]:
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/"

*csv con indicazione colonna indice*



In [ ]:
df = pd.read_csv(BASE_URL + "shirts_example.csv", index_col=0)
df.head()

*tsv con indicazione colonna indice*

In [ ]:
df = pd.read_csv(BASE_URL + "shirts_example.tsv", index_col=0, sep="\t")
df.head()

*json strutturato in formato tabellare*

In [ ]:
df = pd.read_json(BASE_URL + "shirts_example.json")
df.head(10)

*xml con elementi "row" indicati esplicitamente*

In [ ]:
df = pd.read_xml(BASE_URL + "shirts_example.xml", xpath='.//row')
df.head()

*html con unica table con indicazione indice e riga di header*

In [ ]:
df = pd.read_html(BASE_URL + "shirts_example.html", index_col=0, header = 0)
type(df) # lista di tutte le tabelle
type(df[0])
df[0].head()

*excel office oppure openoffice*

In [ ]:
pip install odfpy

In [ ]:
df = pd.read_excel(BASE_URL + "shirts_example.ods", index_col=0) #header=None per non usare prima riga come header
df.head()

## Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/"

*Ordinal encoding su variabile qualitativa*

Assegnazione di valore numero ordinato da 1

*  pandas

In [ ]:
df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

size_mapping = {"S":1, "M":2, "L":3, "XL":4}
type(df["taglia"] ) #series of dataframe
df["taglia"] = df["taglia"].map(size_mapping)
df.head()

*  numpy

In [ ]:
df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

X = df.values #array numpy multidimensionale (100x3)
X.shape

size_mapping = {"S":1, "M":2, "L":3, "XL":4}
fmap = np.vectorize(lambda t:size_mapping[t]) #funzione vettorizzata da applicare ad array numpy
X[:,0] = fmap(X[:,0])
X[:5,:]

*One hot encoding su variabili categoriche*

Aggiunta di colonna true/false per ogni possibile valore

*  pandas

In [ ]:
df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

df = pd.get_dummies(df, columns=["colore"], prefix="color", prefix_sep="-") # "nomeColonnaOriginale_Classe" di default
df.head()

*  numpy con scikit-learn

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

X = df.values #array numpy multidimensionale (100x3)
X.shape

transf = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(), [1]) #tupla identificativa, ultimo parametro è la lista delle colonne
    ],
    remainder = "passthrough" #per mantenere colonne non modificate
)

X = transf.fit_transform(X) #array numpy trasformato
X

*Label encoding*

Codificare variabile target che si presenta come stringa


*   pandas (come ordinal encoding sopra, assegnando valori 1 e 0)

In [ ]:
df = pd.read_csv(BASE_URL+"shirts_sold.csv", index_col=0)

size_mapping = {"SI":1, "NO":0}
df["venduta"] = df["venduta"].map(size_mapping)
df.head()

*  numpy

In [ ]:
df = pd.read_csv(BASE_URL+"shirts_sold.csv", index_col=0)
df.head()

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["venduta"] = le.fit_transform(df["venduta"])
print(df.head())

le.classes_ #classi usate

# è possibile applicare trasformazione inversa...
y = [0,1,0,0,1]
y = le.inverse_transform(y)
print(y)

*Drop per dati mancanti*

In [ ]:
df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)
df.head()

df.count() # numero colonne mancano su sepal_width e petal_width
print(df.isna().sum()) # numero valori NA per colonna
print(df.shape) # numero osservazioni iniziale

#rimozione per riga...
df_drop = df.copy()
df_drop = df_drop.dropna()
df_drop.head()
print(df_drop.shape)

# - usare "how" = all per rimuovere righe con tutte colonne vuote...
# - usare "subset" = ... per rimuovere righe quella colonna vuote...
df_drop = df_drop.dropna(subset=["sepal_width"])

#rimozione per colonna, possibile indicare soglia minima...
df_drop = df.copy()
df_drop = df_drop.dropna(axis=1, thresh=145)
print(df_drop.shape)

# - per calcolare percentuale al 90% del numero osservazioni:
thresh = int(df.shape[0]*0.9)

*Imputazione dei dati feature mancanti, cioè sostituire NA (media, mediana, moda)*

*  pandas (su singola colonna)

In [ ]:
df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)
df.head()

df_imp = df.copy()

col = "sepal_width"
replace_with = round(df_imp[col].mean(), 1)
# - si poteva fare anche con la median
# - si poteva fare anche con la moda, prendendo il più frequente
#   replace_with = round(df_imp[col].mode()[0], 1)

df_imp[col] = df_imp[col].fillna(replace_with)
df_imp.head()
print(df_imp.isna().sum()) # ora gli NA sono zero per la colonna

*  pandas (su più colonne)

In [ ]:
df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)
df.head()

df_imp = df.copy()

replace_with = round(df_imp.mean(numeric_only=True), 1) #series con valori medi per colonna
df_imp = df_imp.fillna(replace_with)
df_imp.head()
print(df_imp.isna().sum()) # tutti gli NA sono zero

*  numpy e scikit-learn

In [ ]:
from sklearn.impute import SimpleImputer

df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)

X = df.drop("species", axis=1).values #si rimuove variabile target, che non va adeguata..
X.shape

imp = SimpleImputer(strategy="mean")
X_imp = imp.fit_transform(X)
X_imp[:5]
np.isnan(X).sum(axis=0)
np.isnan(X_imp).sum(axis=0) #valori NaN per asse colonna

*Normalizzazione*

x-min(x) / max(x) - min(x)

*  pandas

In [ ]:
df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
df.head()

df_norm = df.copy()
features = ["alcol", "flavonoidi"] #escludiamo il target che è classe, che non ha senso normalizzare
to_norm = df_norm[features]
df_norm[features] = (to_norm-to_norm.min()) / (to_norm.max()-to_norm.min())
df_norm.head() #così abbiamo tutto tra min 0 e max 1

*  numpy e scikit-learn

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
X = df.drop("classe", axis=1).values #rimuoviamo colonna target
X.shape

mms = MinMaxScaler()
X_norm = mms.fit_transform(X)
X_norm[:5] #così abbiamo tutto tra min 0 e max 1

*Standardizzazione*

x-mean(x) / sd(x)

*  pandas

In [67]:
df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
df.head()

df_std = df.copy()
features = ["alcol", "flavonoidi"] #escludiamo il target che è classe, che non ha senso standardizzare
to_std = df_std[features]

df_std[features] = (to_std - to_std.mean()) / to_std.std(ddof=0) #ddof per non fare -1 (std campionaria) ma della popolazione (come numpy)
df_std.head() #infatti la media è quasi uguale a zero e la deviazione standard prossima a 1

,classe,alcol,flavonoidi
0,1,1.514341,1.031908
1,1,0.245597,0.731565
2,1,0.196325,1.212114
3,1,1.686791,1.462399
4,1,0.294868,0.661485


*  scikit-learn

In [71]:
from sklearn.preprocessing import StandardScaler

df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
X = df.drop("classe", axis=1).values #rimuoviamo colonna target
X.shape

ss = StandardScaler()
X_std = ss.fit_transform(X)
X_std[:5]
print(X_std.mean()) # prossimo zero
print(X_std.std()) # prossimo a 1

-6.187310339484019e-16
1.0
